In [14]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import re
import os

In [2]:
from huggingface_hub import hf_hub_download
import fasttext

filename = hf_hub_download(
    repo_id="mesolitica/fasttext-language-detection-bahasa-en", 
    filename="fasttext.ftz"
)
lang_model = fasttext.load_model(filename)

In [3]:
df = pd.read_parquet('dialects_segment-00000-of-00001.parquet')
df.head()

,new_text,audio_filename
0,<|startoftranscript|><|ms|><|transcribe|><|0.0...,dialects_processed/KILAUAN EMAS ZON UTARA KEP...
1,<|startoftranscript|><|ms|><|transcribe|><|0.0...,dialects_processed/KILAUAN EMAS ZON UTARA KEP...
2,<|startoftranscript|><|ms|><|transcribe|><|0.0...,dialects_processed/KILAUAN EMAS ZON UTARA KEP...
3,<|startoftranscript|><|ms|><|transcribe|><|0.0...,dialects_processed/KILAUAN EMAS ZON UTARA KEP...
4,<|startoftranscript|><|ms|><|transcribe|><|0.0...,dialects_processed/KILAUAN EMAS ZON UTARA KEP...


In [4]:
re.sub(r"<\|.*?\|>", "", df['new_text'].iloc[0]).strip()

'Abang yakin tak, untuk mewakili Zon Utara?'

In [5]:
lang_model.predict(re.sub(r"<\|.*?\|>", "", df['new_text'].iloc[0]).strip())[0][0]

'__label__bahasa'

In [6]:
filtered = []
for i in tqdm(range(len(df))):
    t = re.sub(r"<\|.*?\|>", "", df['new_text'].iloc[i]).strip()
    if lang_model.predict(t)[0][0] == '__label__bahasa':
        filtered.append(df.iloc[i].to_dict())

100%|██████████████████████████████████████████████████████████████████████████| 1131225/1131225 [00:28<00:00, 40217.32it/s]


In [7]:
len(filtered)

914681

In [19]:
import random

selected = random.sample(filtered, 100000)

In [20]:
for i in range(len(selected)):
    selected[i]['audio_filename'] = os.path.join('/home/husein/ssd3', selected[i]['audio_filename'])

In [21]:
import json

with open('filter-dialects.json', 'w') as fopen:
    json.dump(selected, fopen)

In [22]:
selected[0]

{'new_text': '<|startoftranscript|><|ms|><|transcribe|><|0.12|> yang ingin diberikan kepada anda,<|1.42|><|1.78|> istimewa untuk Lazada Chief Discount Officer 99 ini,<|5.46|><|endoftext|>',
 'audio_filename': '/home/husein/ssd3/dialects_processed/[LIVE] Lazada Chief Discount Officer [azpcP3mfGyo]/[LIVE] Lazada Chief Discount Officer [azpcP3mfGyo]_194.mp3'}

In [23]:
import IPython.display as ipd
ipd.Audio(selected[0]['audio_filename'])